In [ ]:
pip install bert-extractive-summarizer

In [ ]:
from summarizer import Summarizer
text = open("article.txt").read()
model = Summarizer('distilbert-base-uncased')
result = model(text, num_sentences=3)
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Amid the political turmoil of outgoing British Prime Minister Liz Truss’s
short-lived government, the Bank of England has found itself in the
fiscal-financial crossfire. But wherefore it was that after having repeatedly smelt the sea
as a merchant sailor, I should now take it into my head to go on a
whaling voyage; this the invisible police officer of the Fates, who
has the constant surveillance of me, and secretly dogs me, and
influences me in some unaccountable way--he can better answer than
any one else. The landlord was near spraining his wrist, and I told him for
heaven's sake to quit--the bed was soft enough to suit me, and I did
not know how all the planing in the world could make eider down of a
pine plank.


**Text Generator**

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np

# Load pre-trained BERT model and tokenizer from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Hyperparameters
MAX_LEN = 20  # Max sequence length for input and output
VOCAB_SIZE = tokenizer.vocab_size  # ~30,522 for BERT base
EMBEDDING_DIM = 768  # BERT's hidden size

# Define the generative model
def build_generator():
    input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

    # Get BERT embeddings
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask)[0]  # [batch, seq_len, 768]

    # Add a dense layer for token prediction
    outputs = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')(bert_outputs)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

# Build and compile the model
generator = build_generator()
generator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy')

# Preprocess input prompt
def preprocess_prompt(prompt, max_len=MAX_LEN):
    encoding = tokenizer(prompt, max_length=max_len, padding='max_length',
                         truncation=True, return_tensors='tf')
    return encoding['input_ids'], encoding['attention_mask']

# Generate text
def generate_text(prompt, max_tokens=10, temperature=1.0):
    input_ids, attention_mask = preprocess_prompt(prompt)
    generated_ids = input_ids.numpy().tolist()[0]  # Start with input_ids

    for _ in range(max_tokens):
        # Predict next token probabilities
        preds = generator.predict([input_ids, attention_mask], verbose=0)  # [1, MAX_LEN, VOCAB_SIZE]

        # Use the last valid position if sequence exceeds MAX_LEN
        position = min(len(generated_ids) - 1, MAX_LEN - 1)
        next_token_logits = preds[0, position, :] / temperature  # Apply temperature

        # Sample the next token
        next_token = tf.random.categorical(tf.expand_dims(next_token_logits, 0), 1)[0, 0].numpy()

        # Append the predicted token
        generated_ids.append(next_token)

        # Update input_ids and attention_mask (truncate to MAX_LEN)
        input_ids = tf.constant([generated_ids[-MAX_LEN:]], dtype=tf.int32)  # Keep last MAX_LEN tokens
        attention_mask = tf.ones_like(input_ids, dtype=tf.int32)

        # Stop if end token ([SEP]) is generated
        if next_token == tokenizer.sep_token_id:
            break

    # Decode the generated token IDs to text
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "The future of AI is"
generated = generate_text(prompt, max_tokens=10, temperature=0.7)
print(f"Generated text: {generated}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Generated text: the future of ai is flowering participants buzz 1944 suspense worcestershire google bottom vertically hall


In [ ]:
!pip install transformers torch datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# DISABLE W&B LOGGING
import os
os.environ["WANDB_DISABLED"] = "true"

# Now run your BERT fine-tuning code
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from datasets import load_dataset
import evaluate
import numpy as np

# Load dataset
dataset = load_dataset("imdb")

# Take smaller subsets for faster training
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
eval_dataset = dataset["test"].shuffle(seed=42).select(range(200))

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Set up evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Configure training (WITHOUT W&B)
training_args = TrainingArguments(
    output_dir="./bert_imdb_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # Explicitly tell Trainer not to use W&B
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

# Train without W&B
trainer.train()

# Save model
model.save_pretrained("./bert_imdb_model")
tokenizer.save_pretrained("./bert_imdb_model")

# Test predictions
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
print(classifier("This movie was great!"))

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.352200,0.334855,0.870000
2,0.178300,0.422719,0.870000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.352200,0.334855,0.870000
2,0.178300,0.422719,0.870000
3,0.158400,0.495052,0.875000


Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9455056190490723}]
